In [5]:
import pandas as pd
import re

# Load your MIMIC-IV discharge notes dataset
df = pd.read_csv('data/training_text_drg.csv')

In [2]:
import pandas as pd
from tqdm.auto import tqdm
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import torch

# Check for MPS availability and set device accordingly
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
print('device', device)

# Load the tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")
model = AutoModelForTokenClassification.from_pretrained("d4data/biomedical-ner-all")
model.to(device)


device mps


DistilBertForTokenClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
    

In [55]:

# Create the NER pipeline
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple", device=device)

# Function to split text into chunks of a specified maximum length
def split_text_into_chunks(text, max_length=512):
    tokens = tokenizer.tokenize(text)
    chunks = []
    for i in range(0, len(tokens), max_length):
        chunk = tokens[i:i + max_length]
        chunks.append(tokenizer.convert_tokens_to_string(chunk))
    return chunks

# Define a function to process text in batches with chunking
def process_in_batches(text_list, batch_size=32, max_length=512):
    all_ner_results = []
    for text in tqdm(text_list, desc="Processing Texts"):
        ner_results = []
        chunks = split_text_into_chunks(text, max_length=max_length)
        for i in range(0, len(chunks), batch_size):
            batch = chunks[i:i + batch_size]
            ner_results.extend(ner_pipeline(batch))
        all_ner_results.append(ner_results)
    return all_ner_results

# Load your MIMIC-IV discharge notes dataset
df = pd.read_csv('data/training_text_drg.csv')

df_test = df.iloc[:10].copy()

# Apply the batch processing function
texts = df_test['training_text'].tolist()
df_test['ner_entities'] = process_in_batches(texts)

# Save the processed DataFrame
df_test.to_csv('processed_data_drg_test.csv', index=False)

Processing Texts:   0%|          | 0/10 [00:00<?, ?it/s]

In [56]:
len(df_test.iloc[0]['ner_entities'])

6

In [59]:
df_test.iloc[0]['ner_entities'][0][6:15]

[{'entity_group': 'Sign_symptom',
  'score': 0.99995184,
  'word': 'd',
  'start': 403,
  'end': 404},
 {'entity_group': 'Sign_symptom',
  'score': 0.99967515,
  'word': '##ys',
  'start': 404,
  'end': 406},
 {'entity_group': 'Sign_symptom',
  'score': 0.8531294,
  'word': '##pnea',
  'start': 406,
  'end': 410},
 {'entity_group': 'Diagnostic_procedure',
  'score': 0.7449479,
  'word': 'cad',
  'start': 528,
  'end': 531},
 {'entity_group': 'Diagnostic_procedure',
  'score': 0.41368252,
  'word': 'p',
  'start': 536,
  'end': 537},
 {'entity_group': 'Diagnostic_procedure',
  'score': 0.2637567,
  'word': 'des',
  'start': 538,
  'end': 541},
 {'entity_group': 'Detailed_description',
  'score': 0.99993813,
  'word': 'is',
  'start': 547,
  'end': 549},
 {'entity_group': 'Detailed_description',
  'score': 0.99299026,
  'word': '##chemic',
  'start': 549,
  'end': 555},
 {'entity_group': 'Sign_symptom',
  'score': 0.95868415,
  'word': 'mr',
  'start': 556,
  'end': 558}]

In [36]:
text = texts[0]
tokens = tokenizer.tokenize(text)
print(len(tokens))
chunks = split_text_into_chunks(text)
for chunk in chunks:
    chunk_tokens = tokenizer.tokenize(chunk)
    print(f'[Num chars: {len(chunk)} Num tokens: {len(chunk_tokens)}] {chunk[:50]}')

2917
[Num chars: 1859 Num tokens: 512] admission type : ew emer., admission location : em
[Num chars: 1721 Num tokens: 512] , with lad, twi in the inferior leads which appear
[Num chars: 1330 Num tokens: 512] . conjunctiva were pink, no pallor or cyanosis of 
[Num chars: 1909 Num tokens: 512] mcv - 95 mch - 30. 6 mchc - 32. 2 rdw - 16. 0 * rd
[Num chars: 1952 Num tokens: 512] with close pcp and _ _ _ to monitor weights and bl
[Num chars: 1186 Num tokens: 357] : increased nifedipine cr to 60mg bid given elevat


In [37]:
text[:100]

'Admission Type: EW EMER., Admission Location: EMERGENCY ROOM, Insurance: Medicare, Language: ENGLISH'

In [13]:
df_test

,hadm_id,CARD_ARRHYTHMIA,CELL_SKIN_INF,CELL_NO_MCC,CVA_INFARCT,PANCREAS_DIS,DIGEST_DIS_NO_MCC,HEART_FAILURE,HEART_SHOCK_MCC,KIDNEY_UTI,JOINT_REPLACE_NO_MCC,DIGEST_DIS_OTHER,GASTRO_NAUSEA,PNEUMONIA_OTHER,PCI_NO_AMI,PSYCHOSES,SEPTICEMIA,SYNCOPE,training_text,ner_entities
0,29659838,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,"Admission Type: EW EMER., Admission Location: ...","[[{'entity_group': 'History', 'score': 0.89702..."
1,20897796,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,"Admission Type: OBSERVATION ADMIT, Admission L...","[[{'entity_group': 'History', 'score': 0.94635..."
2,21975601,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,"Admission Type: EW EMER., Admission Location: ...","[[{'entity_group': 'History', 'score': 0.93111..."
3,28994087,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,"Admission Type: EW EMER., Admission Location: ...","[[{'entity_group': 'History', 'score': 0.95380..."
4,26295318,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,"Admission Type: URGENT, Admission Location: TR...","[[{'entity_group': 'Clinical_event', 'score': ..."
5,22002850,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,"Admission Type: EW EMER., Admission Location: ...","[[{'entity_group': 'History', 'score': 0.74439..."
6,28301173,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,"Admission Type: URGENT, Admission Location: IN...","[[{'entity_group': 'Clinical_event', 'score': ..."
7,21404960,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,"Admission Type: EW EMER., Admission Location: ...","[[{'entity_group': 'History', 'score': 0.92561..."
8,29460260,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,"Admission Type: EW EMER., Admission Location: ...","[[{'entity_group': 'History', 'score': 0.56675..."
9,26475031,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,"Admission Type: EW EMER., Admission Location: ...","[[{'entity_group': 'History', 'score': 0.78694..."


In [40]:
df_test.iloc[0]['ner_entities']

[[{'entity_group': 'History',
   'score': 0.8970286,
   'word': 'marital',
   'start': 102,
   'end': 109},
  {'entity_group': 'History',
   'score': 0.9263247,
   'word': 'married',
   'start': 118,
   'end': 125},
  {'entity_group': 'History',
   'score': 0.8146181,
   'word': 'race',
   'start': 127,
   'end': 131},
  {'entity_group': 'History',
   'score': 0.33758128,
   'word': 'black',
   'start': 133,
   'end': 138},
  {'entity_group': 'Personal_background',
   'score': 0.57014996,
   'word': 'african',
   'start': 139,
   'end': 146},
  {'entity_group': 'Sign_symptom',
   'score': 0.99868184,
   'word': 'adverse drug reactions',
   'start': 315,
   'end': 337},
  {'entity_group': 'Sign_symptom',
   'score': 0.99995184,
   'word': 'd',
   'start': 371,
   'end': 372},
  {'entity_group': 'Sign_symptom',
   'score': 0.99967515,
   'word': '##ys',
   'start': 372,
   'end': 374},
  {'entity_group': 'Sign_symptom',
   'score': 0.8531294,
   'word': '##pnea',
   'start': 374,
   'end

In [76]:
text[:512]

'Admission Type: EW EMER., Admission Location: EMERGENCY ROOM, Insurance: Medicare, Language: ENGLISH, Marital Status: MARRIED, Race: BLACK/AFRICAN AMERICAN, Gender: F, Age: 73  \nName:  ___          Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergies: \nNo Known Allergies / Adverse Drug Reactions\n \nAttending: ___. Chief Complaint: dyspnea on exertion Major Surgical or Invasive Procedure: None History of Present Ill'

In [60]:
for entity_list in df_test.iloc[0]['ner_entities']:
    i = 0
    for entity in entity_list:
        print(entity)
        i += 1
        if i > 3: break

{'entity_group': 'History', 'score': 0.8970286, 'word': 'marital', 'start': 106, 'end': 113}
{'entity_group': 'History', 'score': 0.9263247, 'word': 'married', 'start': 123, 'end': 130}
{'entity_group': 'History', 'score': 0.8146181, 'word': 'race', 'start': 132, 'end': 136}
{'entity_group': 'History', 'score': 0.33758128, 'word': 'black', 'start': 139, 'end': 144}
{'entity_group': 'Diagnostic_procedure', 'score': 0.99888366, 'word': 't', 'start': 12, 'end': 13}
{'entity_group': 'Diagnostic_procedure', 'score': 0.9852722, 'word': '##wi', 'start': 13, 'end': 15}
{'entity_group': 'Diagnostic_procedure', 'score': 0.9998057, 'word': 'inferior leads', 'start': 23, 'end': 37}
{'entity_group': 'Sign_symptom', 'score': 0.75722, 'word': 'improved', 'start': 118, 'end': 126}
{'entity_group': 'Biological_structure', 'score': 0.99852574, 'word': 'conjunctiva', 'start': 2, 'end': 13}
{'entity_group': 'Sign_symptom', 'score': 0.9998093, 'word': 'pallor', 'start': 28, 'end': 34}
{'entity_group': 'Sig

In [66]:
def merge_entities(entities):
    merged_entities = []
    current_entity = None

    for entity in entities:
        if entity['word'].startswith('##'):
            if current_entity and current_entity['entity_group'] == entity['entity_group']:
                current_entity['word'] += entity['word'][2:]
                current_entity['end'] = entity['end']
                current_entity['score'] = max(current_entity['score'], entity['score'])
            else:
                current_entity = entity
        else:
            if current_entity:
                merged_entities.append(current_entity)
            current_entity = entity
    
    if current_entity:
        merged_entities.append(current_entity)
    
    return merged_entities

# Example usage with the first row of entities
entities = [entity for sublist in df_test['ner_entities'].iloc[0] for entity in sublist]
merged_entities = merge_entities(entities)

# Print out some of the merged entities
for entity in merged_entities[:5]:
    print(entity)


{'entity_group': 'History', 'score': 0.8970286, 'word': 'marital', 'start': 106, 'end': 113}
{'entity_group': 'History', 'score': 0.9263247, 'word': 'married', 'start': 123, 'end': 130}
{'entity_group': 'History', 'score': 0.8146181, 'word': 'race', 'start': 132, 'end': 136}
{'entity_group': 'History', 'score': 0.33758128, 'word': 'black', 'start': 139, 'end': 144}
{'entity_group': 'Personal_background', 'score': 0.57014996, 'word': 'african', 'start': 147, 'end': 154}


In [67]:
merged_entities

[{'entity_group': 'History',
  'score': 0.8970286,
  'word': 'marital',
  'start': 106,
  'end': 113},
 {'entity_group': 'History',
  'score': 0.9263247,
  'word': 'married',
  'start': 123,
  'end': 130},
 {'entity_group': 'History',
  'score': 0.8146181,
  'word': 'race',
  'start': 132,
  'end': 136},
 {'entity_group': 'History',
  'score': 0.33758128,
  'word': 'black',
  'start': 139,
  'end': 144},
 {'entity_group': 'Personal_background',
  'score': 0.57014996,
  'word': 'african',
  'start': 147,
  'end': 154},
 {'entity_group': 'Sign_symptom',
  'score': 0.99868184,
  'word': 'adverse drug reactions',
  'start': 343,
  'end': 365},
 {'entity_group': 'Sign_symptom',
  'score': 0.99995184,
  'word': 'dyspneayspnea',
  'start': 403,
  'end': 410},
 {'entity_group': 'Diagnostic_procedure',
  'score': 0.7449479,
  'word': 'cad',
  'start': 528,
  'end': 531},
 {'entity_group': 'Diagnostic_procedure',
  'score': 0.41368252,
  'word': 'p',
  'start': 536,
  'end': 537},
 {'entity_grou

In [62]:
import json 

# Function to convert pipeline output to a JSON format
def format_entities_to_json(text, entities):
    annotated_text = []
    start_idx = 0

    for entity in entities:
        # Append text before the entity
        if start_idx < entity['start']:
            annotated_text.append({
                'text': text[start_idx:entity['start']],
                'label': 'O'
            })
        
        # Append the entity
        annotated_text.append({
            'text': text[entity['start']:entity['end']],
            'label': entity['entity_group']
        })
        
        start_idx = entity['end']
    
    # Append remaining text if any
    if start_idx < len(text):
        annotated_text.append({
            'text': text[start_idx:],
            'label': 'O'
        })
    
    return annotated_text

# Convert merged entities to JSON format
i = 0
text = df_test['training_text'].iloc[i]
formatted_entities = format_entities_to_json(text, merged_entities)
json_output = json.dumps(formatted_entities, indent=2)
print(json_output)


[
  {
    "text": "Admission Type: EW EMER., Admission Location: EMERGENCY ROOM, Insurance: Medicare, Language: ENGLISH, Mari",
    "label": "O"
  },
  {
    "text": "tal Sta",
    "label": "History"
  },
  {
    "text": "tus: MARRI",
    "label": "O"
  },
  {
    "text": "ED, Rac",
    "label": "History"
  },
  {
    "text": "e:",
    "label": "O"
  },
  {
    "text": " BLA",
    "label": "History"
  },
  {
    "text": "CK/",
    "label": "O"
  },
  {
    "text": "AFRIC",
    "label": "History"
  },
  {
    "text": "AN ",
    "label": "O"
  },
  {
    "text": "AMERICA",
    "label": "Personal_background"
  },
  {
    "text": "N, Gender: F, Age: 73  \nName:  ___          Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergi",
    "label": "O"
  },
  {
    "text": "es: \nNo Known Allergie",
    "label": "Sign_symptom"
  },
  {
    "text": "s / Adverse Drug Reactions\n \nAttending",
    "l

In [69]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("d4data/biomedical-ner-all")

def merge_subword_tokens(tokens):
    merged_tokens = []
    current_token = ""
    
    for token in tokens:
        if token.startswith("##"):
            current_token += token[2:]
        else:
            if current_token:
                merged_tokens.append(current_token)
            current_token = token
    
    if current_token:
        merged_tokens.append(current_token)
    
    return merged_tokens

def visualize_entities_on_tokens(text, entities, tokenizer):
    tokens = tokenizer.tokenize(text)
    merged_tokens = merge_subword_tokens(tokens)
    
    token_labels = ['O'] * len(merged_tokens)
    token_positions = tokenizer.convert_tokens_to_ids(merged_tokens)
    
    for entity in entities:
        start_token = entity['start']
        end_token = entity['end']
        for i, token in enumerate(merged_tokens):
            if token_positions[i] >= start_token and token_positions[i] < end_token:
                token_labels[i] = entity['entity_group']
    
    html = ""
    colors = {
        "History": "#e6f7ff",
        "Sex": "#ffcccc",
        "Clinical_event": "#d9f2e6",
        "Sign_symptom": "#ffffcc",
        "Lab_value": "#e6e6ff",
        "Diagnostic_procedure": "#ffe6cc",
        "Detailed_description": "#ffccf2",
        "Disease_disorder": "#cce6ff",
        "Therapeutic_procedure": "#ffb3b3",
        "Biological_structure": "#e6ffe6",
        "Coreference": "#ffebcc",
        "Date": "#ffcccc",
        "Medication": "#cce6ff",
        "O": "#ffffff"
    }

    for token, label in zip(merged_tokens, token_labels):
        color = colors.get(label, "#ffffff")
        html += f"<span style='background-color:{color}'>{token}</span> "
    
    display(HTML(html))

# Example usage
i = 0
text = df_test['training_text'].iloc[i]
visualize_entities_on_tokens(text, merged_entities, tokenizer)


Token indices sequence length is longer than the specified maximum sequence length for this model (2917 > 512). Running this sequence through the model will result in indexing errors


In [73]:
tokens = tokenizer.tokenize(text)
merged_tokens = merge_subword_tokens(tokens)
' '.join(merged_tokens)

"admission type : ew emer . , admission location : emergency room , insurance : medicare , language : english , marital status : married , race : black / african american , gender : f , age : 73 name : _ _ _ unit no : _ _ _ admission date : _ _ _ discharge date : _ _ _ date of birth : _ _ _ sex : f service : medicine allergies : no known allergies / adverse drug reactions attending : _ _ _ . chief complaint : dyspnea on exertion major surgical or invasive procedure : none history of present illness : _ _ _ with a history of of htn , cad s / p des with ischemic mr and systolic dysfunction , _ _ _ on torsemide , hx of dvt , who presents with 4 days of dyspnea on exertion , leg swelling , and 10 weight gain . of note , patient was seen in the heart failure clinic with dr . _ _ _ on _ _ _ where she noted that she has had persistent dyspnea on exertion and pnd after a lengthy prior hospitalization for dvt / gib . at that time she was started on 40mg po torsemide which initially improved her

In [74]:
tokens

['admission',
 'type',
 ':',
 'e',
 '##w',
 'em',
 '##er',
 '.',
 ',',
 'admission',
 'location',
 ':',
 'emergency',
 'room',
 ',',
 'insurance',
 ':',
 'medicare',
 ',',
 'language',
 ':',
 'english',
 ',',
 'marital',
 'status',
 ':',
 'married',
 ',',
 'race',
 ':',
 'black',
 '/',
 'african',
 'american',
 ',',
 'gender',
 ':',
 'f',
 ',',
 'age',
 ':',
 '73',
 'name',
 ':',
 '_',
 '_',
 '_',
 'unit',
 'no',
 ':',
 '_',
 '_',
 '_',
 'admission',
 'date',
 ':',
 '_',
 '_',
 '_',
 'discharge',
 'date',
 ':',
 '_',
 '_',
 '_',
 'date',
 'of',
 'birth',
 ':',
 '_',
 '_',
 '_',
 'sex',
 ':',
 'f',
 'service',
 ':',
 'medicine',
 'all',
 '##er',
 '##gies',
 ':',
 'no',
 'known',
 'all',
 '##er',
 '##gies',
 '/',
 'adverse',
 'drug',
 'reactions',
 'attending',
 ':',
 '_',
 '_',
 '_',
 '.',
 'chief',
 'complaint',
 ':',
 'd',
 '##ys',
 '##p',
 '##nea',
 'on',
 'ex',
 '##ert',
 '##ion',
 'major',
 'surgical',
 'or',
 'invasive',
 'procedure',
 ':',
 'none',
 'history',
 'of',
 'present',


In [50]:
entities[5:]

[{'entity_group': 'Sign_symptom',
  'score': 0.99868184,
  'word': 'adverse drug reactions',
  'start': 315,
  'end': 337},
 {'entity_group': 'Sign_symptom',
  'score': 0.99995184,
  'word': 'dyspneayspnea',
  'start': 371,
  'end': 378},
 {'entity_group': 'Sign_symptom',
  'score': 0.99967515,
  'word': '##ys',
  'start': 372,
  'end': 374},
 {'entity_group': 'Sign_symptom',
  'score': 0.8531294,
  'word': '##pnea',
  'start': 374,
  'end': 378},
 {'entity_group': 'Diagnostic_procedure',
  'score': 0.7449479,
  'word': 'cad',
  'start': 492,
  'end': 495},
 {'entity_group': 'Diagnostic_procedure',
  'score': 0.41368252,
  'word': 'p',
  'start': 498,
  'end': 499},
 {'entity_group': 'Diagnostic_procedure',
  'score': 0.2637567,
  'word': 'des',
  'start': 500,
  'end': 503},
 {'entity_group': 'Detailed_description',
  'score': 0.99993813,
  'word': 'ischemicchemic',
  'start': 509,
  'end': 517},
 {'entity_group': 'Detailed_description',
  'score': 0.99299026,
  'word': '##chemic',
  

In [15]:
import json


# Function to convert pipeline output to a JSON format
def format_entities_to_json(text, entities):
    annotated_text = []
    start_idx = 0

    for entity in entities:
        # Append text before the entity
        if start_idx < entity['start']:
            annotated_text.append({
                'text': text[start_idx:entity['start']],
                'label': 'O'
            })
        
        # Append the entity
        annotated_text.append({
            'text': text[entity['start']:entity['end']],
            'label': entity['entity_group']
        })
        
        start_idx = entity['end']
    
    # Append remaining text if any
    if start_idx < len(text):
        annotated_text.append({
            'text': text[start_idx:],
            'label': 'O'
        })
    
    return annotated_text

# Convert entities to JSON format
i = 0

# # Flatten the list of lists for the current text
# flat_entities = [entity for sublist in df_test['ner_entities'].iloc[i] for entity in sublist]

formatted_entities = format_entities_to_json(df_test['training_text'].iloc[i], flat_entities)
json_output = json.dumps(formatted_entities, indent=2)
print(json_output)


[
  {
    "text": "Admission Type: EW EMER., Admission Location: EMERGENCY ROOM, Insurance: Medicare, Language: ENGLISH, Mari",
    "label": "O"
  },
  {
    "text": "tal Sta",
    "label": "History"
  },
  {
    "text": "tus: MARRI",
    "label": "O"
  },
  {
    "text": "ED, Rac",
    "label": "History"
  },
  {
    "text": "e:",
    "label": "O"
  },
  {
    "text": " BLA",
    "label": "History"
  },
  {
    "text": "CK/",
    "label": "O"
  },
  {
    "text": "AFRIC",
    "label": "History"
  },
  {
    "text": "AN ",
    "label": "O"
  },
  {
    "text": "AMERICA",
    "label": "Personal_background"
  },
  {
    "text": "N, Gender: F, Age: 73  \nName:  ___          Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: MEDICINE\n \nAllergi",
    "label": "O"
  },
  {
    "text": "es: \nNo Known Allergie",
    "label": "Sign_symptom"
  },
  {
    "text": "s / Adverse Drug Reactions\n \nAttending",
    "l

In [17]:
from IPython.core.display import display, HTML

# Function to visualize annotated text
def visualize_annotated_text(annotated_text):
    html = ""
    colors = {
        "Age": "#e6f7ff",
        "Sex": "#ffcccc",
        "Clinical_event": "#d9f2e6",
        "Sign_symptom": "#ffffcc",
        "Lab_value": "#e6e6ff",
        "Diagnostic_procedure": "#ffe6cc",
        "Detailed_description": "#ffccf2",
        "Disease_disorder": "#cce6ff",
        "Therapeutic_procedure": "#ffb3b3",
        "Biological_structure": "#e6ffe6",
        "Coreference": "#ffebcc",
        "Date": "#ffcccc",
        "Medication": "#cce6ff"
    }
    
    for segment in annotated_text:
        color = colors.get(segment['label'], "#ffffff")
        html += f"<span style='background-color:{color}'>{segment['text']}</span>"
    
    display(HTML(html))

# Visualize the annotated text
visualize_annotated_text(formatted_entities)

/var/folders/dv/lndb9rrs2kv2v5g8cyfyx0wr0000gn/T/ipykernel_75765/2602753649.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML
